<font size="18">Image Augmentation App</font>

This application demonstrates a simple rocAL pipeline with different augmentations supported by rocAL.

<font size="12"> Common Code </font>

In [ ]:
from amd.rocal.plugin.generic import ROCALClassificationIterator
from amd.rocal.pipeline import Pipeline
import amd.rocal.fn as fn
import amd.rocal.types as types
import matplotlib.pyplot as plt
%matplotlib inline


<font size= "12" >Configuring rocAL pipeline </font>

Configure the pipeline parameters as required by the user.

In [ ]:
data_path = "/media/MIVisionX-data/rocal_data/coco/coco_10_img/train_10images_2017/"
rocal_cpu = True
device = "cpu" 
batch_size =  1
num_threads = 1
random_seed = 1
local_rank =  0
world_size =  1
display = True 

In [ ]:
pipe = Pipeline(batch_size=batch_size, num_threads=num_threads, device_id=local_rank, seed=random_seed, rocal_cpu=rocal_cpu, tensor_layout=types.NHWC , tensor_dtype=types.FLOAT)


<font size="12"> Image augmentation pipeline </font>

Here the file reader is used followed by the turbo jpeg decoder. In this pipeline, cascaded augmentations are added on the decoded images.<br>Multiple augmentation outputs are returned using set_outputs

In [ ]:
with pipe:
        jpegs, _ = fn.readers.file(file_root=data_path)
        images = fn.decoders.image(jpegs, file_root=data_path, output_type=types.RGB, shard_id=0, num_shards=1, random_shuffle=True)
        resize_outputs = fn.resize(images, resize_width=300, resize_height=300)
        brightness_outputs = fn.brightness(resize_outputs)
        contrast_outputs = fn.contrast(resize_outputs)
        fisheye_outputs = fn.fish_eye(resize_outputs)
        fog_outputs = fn.fog(resize_outputs)
        exposure_outputs = fn.exposure(resize_outputs)
        color_twist_outputs = fn.color_twist(resize_outputs)
        saturation_outputs = fn.saturation(resize_outputs)
        pipe.set_outputs(resize_outputs, brightness_outputs, contrast_outputs, fisheye_outputs, fog_outputs, exposure_outputs, color_twist_outputs, saturation_outputs)

In [ ]:
pipe.build()
# Dataloader
data_loader = ROCALClassificationIterator(pipe)

<font size ="12">Visualizing  outputs</font>

The output of augmented images are displayed using imshow()

In [ ]:
cnt = 0
aug_list = ["resize", "brightness", "contrast", "fisheye", "fog", "exposure", "colortwist", "saturation"] 
fig, axes = plt.subplots(nrows=2, ncols=4, figsize=(15,15))
row = 0
col = 0
for i, it in enumerate(data_loader, 0):
    for img in it[0]:
        if(cnt < len(aug_list)):
            axes[row, col].set_title(aug_list[cnt])
        img = (img[0]).astype('uint8')
        axes[row, col].imshow(img)
        cnt += 1
        row += 1
        if(row == 2):
            row = 0
            col += 1
        if(col == 4):
            col = 0
data_loader.reset()